In [1]:
import numpy as np
import GPy
import seaborn as sns
import math
import matplotlib.pyplot as plt
import edward as ed
import tensorflow as tf
# import tensorflow.contrib.eager as tfe
# tfe.enable_eager_execution()
from edward.models import MultivariateNormalFullCovariance, Normal
from scipy.stats import bernoulli
from tensorflow.contrib.distributions import  Bernoulli
from tensorflow.contrib.distributions import Categorical

In [2]:
class RBF():
    
    def __init__(self, input_dim, variance=1., lengthscale=1.):
        self.input_dim = input_dim
        self.variance = variance
        self.lengthscale = lengthscale
        
    def K(self, X, X2):
        return self.variance * tf.exp(-self.square_dist(X, X2) / 2)
        
    def square_dist(self, X, X2):
        X = X / self.lengthscale
        Xs = tf.reduce_sum(tf.square(X), 1)
        if X2 is None:
            return -2 * tf.matmul(X, tf.transpose(X)) + tf.reshape(Xs, (-1, 1)) + tf.reshape(Xs, (1, -1))
        else:
            X2 = X2 / self.lengthscale
            X2s = tf.reduce_sum(tf.square(X2), 1)
            return -2 * tf.matmul(X, tf.transpose(X2)) + tf.reshape(Xs, (-1, 1)) + tf.reshape(X2s, (1, -1))

In [3]:
class Periodic():
    
    def __init__(self, input_dim, period=1.0, variance=1.0, lengthscales=1.0):
        self.input_dim = input_dim
        self.period = period
        self.variance = variance
        self.lengthscales = lengthscales
        
    def K(self, X, X2):
        
        if X2 is None:
            X2 = X
            
        f = tf.expand_dims(X, 1)  # now N x 1 x D
        f2 = tf.expand_dims(X2, 0)  # now 1 x M x D

        r = np.pi * (f - f2) / self.period
        r = tf.reduce_sum(tf.square(tf.sin(r) / self.lengthscales), 2)

        return self.variance * tf.exp(-0.5 * r)

In [4]:
def build_toy_dataset1():
    rate = 2
    measure = 50
    N = np.random.poisson(measure * rate)
    S = np.expand_dims(np.sort(np.linspace(0, measure, N)), axis = 1)
    Z = 2*np.exp(-S/15) + np.exp(-((S-25)/10)**2)
    print(N)
    return S, Z, N, rate, measure

def build_toy_dataset2():
    rate = 12
    measure = 5
    N = np.random.poisson(measure * rate)
    S = np.expand_dims(np.sort(np.linspace(0, measure, N)), axis = 1)
    Z = 5 * np.sin(S**2) + 6
    print(N)
    return S, Z, N, rate, measure

In [5]:
kern = RBF(input_dim = 1, lengthscale = 0.1)
S, Z, N, rate, measure = build_toy_dataset1()
# V = tf.squeeze(1 / (1 + tf.exp(-Z)), axis=-1)
# R = tf.random_uniform([N])
# accept = tf.where(R < V)
# S_k = tf.squeeze(tf.gather(S, accept), axis=-1)
# Z_k = tf.squeeze(tf.gather(Z, accept), axis=-1)
R = np.random.uniform(0, 1, N)
# V = (Z / rate).flatten()
with tf.Session() as sess:
    C = sess.run(kern.K(S,S))
G = np.random.multivariate_normal(np.zeros((S.shape[0])), C)
accept = np.where(R < (1 / (1 + np.exp(-G))))
S_k  = np.take(S, accept, axis=0).reshape(-1,1)
Z_k  = np.take(Z, accept, axis=0).reshape(-1)
#G_k = np.log(Z_k / (rate - Z_k)).reshape(-1)
G_k = np.take(G, accept, axis=0).reshape(-1, 1)#random.multivariate_normal(np.zeros((S_k.shape[0])), k.K(S_k,S_k))
print(G_k)

103
[[ 1.15654771]
 [ 1.41502556]
 [-0.27889562]
 [-0.64665835]
 [-0.12125882]
 [ 1.24664819]
 [-1.0690974 ]
 [-1.34035636]
 [-1.27206654]
 [ 1.48367801]
 [ 0.67363857]
 [-0.23248345]
 [-0.41493631]
 [-0.46698159]
 [ 0.05538056]
 [ 1.83318892]
 [ 0.19708413]
 [ 0.85825365]
 [ 1.12350964]
 [ 0.57085662]
 [-0.58147139]
 [-0.48101529]
 [ 0.05264353]
 [ 1.52192179]
 [-0.34910104]
 [-1.19333334]
 [ 1.16586852]
 [ 0.16262901]
 [-0.32900537]
 [ 0.59067272]
 [ 0.56624843]
 [-0.30544282]
 [ 0.71312829]
 [ 1.21176169]
 [ 0.16295362]
 [-2.16315105]
 [ 1.46290929]
 [ 0.819249  ]
 [-0.22396924]
 [-1.43974062]
 [ 0.15788842]
 [ 1.49597649]
 [-0.90944064]
 [-0.33518519]]


In [11]:
def conditional(x_new, x, y, kernel):

    B = kernel.K(x, x_new)
    A = kernel.K(x_new, x_new)
    X = kernel.K(x, x)
    N = tf.shape(X)[0]
    mu = tf.matmul(B, tf.matmul(tf.matrix_inverse(X + 1e-5*tf.eye(N)), y), transpose_a = True)
    sigma = A - tf.matmul(B, tf.matmul(tf.matrix_inverse(X + 1e-5*tf.eye(N)),B), transpose_a = True)
    return tf.squeeze(mu), tf.squeeze(sigma)

In [7]:
x_new = tf.random_uniform((1,1))
kern = RBF(input_dim=1, lengthscale=0.1)
mu_new, sigma_new = conditional(x_new, S_k, Z_k, kern)
print(mu_new)

TypeError: Input 'b' of 'MatMul' Op has type float32 that does not match type float64 of argument 'a'.

In [12]:
def add_event(x_new, y_new, x_M, y_M):
    x_M = tf.concat([x_M, x_new], 0)
    y_M = tf.concat([y_M, y_new], 0)
    return x_M, y_M
    
def erase_event(x_M, y_M, c):
    
    print(tf.shape(x_M), tf.shape(y_M))
    x_M = tf.concat([tf.slice(x_M, [0,0],[c,1]), tf.slice(x_M, [c+1,0], [-1,1])], 0)
    y_M = tf.concat([tf.slice(y_M, [0,0],[c,1]), tf.slice(y_M, [c+1,0], [-1,1])], 0)
#   x_M = tf.slice(x_M, [0,0],[c-1,1])#, tf.slice(x_M, [c+1,0], [-1,1]))
#   y_M = tf.slice(y_M, [0,0],[c-1,1])#, tf.slice(y_M, [c+1,0], [-1,1]))
    return x_M, y_M

def insert_event(x_K, y_K, x_M, y_M):

    print("Event inserted")
    M = tf.shape(x_M)[0]
    x_new = tf.random_uniform((1,1), minval=0.0, maxval=measure)
    mu_new, sigma_new = conditional(x_new, tf.concat([x_K,x_M], 0), tf.concat([y_K, y_M], 0), kern)
    y_new = tf.random_normal((1,1), mean=mu_new, stddev=tf.sqrt(sigma_new))
    ratio = tf.log(float(rate*measure))
    ratio -= tf.log(tf.cast(M+1, tf.float32))
    ratio -= tf.log(1+tf.exp(y_new))
    a = tf.random_uniform((1,))
    #print(tf.squeeze(tf.less(tf.log(a), ratio)))
    x_M, y_M = tf.cond(tf.squeeze(tf.less(tf.log(a), ratio)), lambda: add_event(x_new, y_new, x_M, y_M), lambda: (x_M, y_M))
    return x_M, y_M

def delete_util(x_M, y_M):
  
    M = tf.shape(x_M)[0]
    c = tf.random_uniform((1,), minval=0, maxval=M, dtype = tf.int32)
    c = tf.squeeze(c)
    ratio = tf.log(tf.cast(M, tf.float32))
    ratio += tf.log(1 + tf.exp(tf.slice(y_M, [c,0], [1,1])))
    ratio -= tf.log(float(rate*measure))
    a = tf.random_uniform((1,))
    
    x_M, y_M = tf.cond(tf.squeeze(tf.less(tf.log(a), ratio)), lambda: erase_event(x_M, y_M, c), lambda: (x_M, y_M))

    return x_M, y_M

def delete_event(x_K, y_K, x_M, y_M):
    print("Event deleted")
    M = tf.shape(x_M)[0]
    x_M, y_M = tf.cond(tf.equal(M, tf.constant(0)), lambda: (x_M, y_M), lambda: delete_util(x_M, y_M))
    return x_M, y_M


In [62]:
#F = tf.Variable(np.zeros((N,1)), name="FunctionVal", dtype=tf.float32)

def optimizeLogPost(G, kern):
    
#     S = tf.concat([x_K, x_M], 0)
#     G = tf.concat([y_K, y_M], 0)
#     D = tf.shape(S)[1]       # Dimension of the space
#     K = tf.shape(x_K)[0]
#     M = tf.shape(x_M)[0]
#     N = K + M
#     C = kernel.K(S,S)
    S = tf.placeholder(dtype=tf.float32, name="S")
    C = kern.K(S,S)
    #C = tf.placeholder(dtype=tf.float32, name="C")
    K = tf.placeholder(dtype=tf.int32, name="K")
    M = tf.placeholder(dtype=tf.int32, name="M")
    F = tf.Variable(np.zeros((1,1)), dtype=tf.float32, validate_shape=False)
    N = K + M
    
    
    
    ## Learn function values via gradient optimization

    #F = tf.Variable(lambda: tf.zeros((N,1)), name="FunctionVal", dtype=tf.float32)
    #tf.Variable(F, lambda: G, validate_shape=False)
    F = tf.Variable(G, name="FunctionVal", dtype=tf.float32)
    #init_OP = tf.global_variables_initializer()

    #print(F)
    #print(F.shape)
    prior_loss = 0.5 * tf.matmul(tf.transpose(F), tf.matmul(tf.matrix_inverse(C + 1e-6*tf.eye(N)), F))
    #print(prior_loss)
    prior_loss = tf.squeeze(prior_loss)
    likelihood_loss = tf.reduce_sum(tf.log(tf.ones([K,1]) + tf.exp(-tf.slice(F, [0,0], [K,1]))))
    likelihood_loss += tf.reduce_sum(tf.log(tf.ones([M,1]) + tf.exp(tf.slice(F, [K,0], [M,1]))))
    #print(likelihood_loss)
    loss = prior_loss + likelihood_loss
    
    
    #print(loss)
    train_op = tf.train.AdadeltaOptimizer(0.1, 0.95, 1e-5).minimize(loss)
    init_OP = tf.global_variables_initializer()
    return init_OP, train_op, F, S, C, K, M

In [65]:

def sample_cond(x_K, y_K, x_M, y_M, i):
    return tf.less(i, tf.shape(x_M)[0])

### Change location at a particular index
def sample_step(x_K, y_K, x_M, y_M, i):
    x_new = tf.random_uniform((1,1))
    mu_new, sigma_new = conditional(x_new, tf.concat([x_K,x_M], 0), tf.concat([y_K, y_M], 0), kern)
    y_new = tf.random_normal((1,1), mean=mu_new, stddev=tf.sqrt(sigma_new))
    ratio = tf.log(1 + tf.exp(y_M[i]))
    ratio -= tf.log(1 + tf.exp(y_new))
    a = tf.random_uniform((1,))
    accept = tf.squeeze(tf.less(tf.log(a), ratio))
    x_M = tf.cond(accept, lambda: tf.concat([tf.slice(x_M, [0,0],[i,1]), tf.concat([x_new, tf.slice(x_M, [i+1,0], [-1,1])], 0)], 0), lambda: x_M)
    y_M = tf.cond(accept, lambda: tf.concat([tf.slice(y_M, [0,0],[i,1]), tf.concat([y_new, tf.slice(y_M, [i+1,0], [-1,1])], 0)], 0), lambda: y_M)
    i = tf.add(i, 1)
    return x_K, y_K, x_M, y_M, i
    
def thinned_cond(x_K, y_K, x_M, y_M, i):
    return tf.less(i, tf.constant(10))

def thinned_step(x_K, y_K, x_M, y_M, i):
    
    bern = Bernoulli(probs = bern_p)
    x_M, y_M = tf.cond(tf.equal(bern.sample(), 1), lambda: insert_event(x_K, y_K, x_M, y_M), lambda: delete_event(x_K, y_K, x_M, y_M))
    i = tf.add(i, 1)
    return x_K, y_K, x_M, y_M, i

def loop_cond(n_iter,it, x_K, y_K, x_M, y_M):
    return tf.less(it, n_iter)

## Loop step
def loop_step(n_iter, ind, x_K, y_K, x_M, y_M):
    
    #print(it)
    # Sample number of thinned events
    i = tf.Variable(0)
    x_K, y_K, x_M, y_M, i = tf.while_loop(thinned_cond, thinned_step, [x_K, y_K, x_M, y_M, i])
    


    print("Thinning completed")
    # Sample thinned locations
    it = tf.Variable(0)
    x_K, y_K, x_M, y_M, it = tf.while_loop(sample_cond, sample_step, [x_K, y_K, x_M, y_M, it])
    
    print("Locations found")

    with tf.Session() as sess:
         sess.run(tf.global_variables_initializer())
         x_K, y_K, x_M, y_M = sess.run([x_K, y_K, x_M, y_M])
    
    print(x_K.shape, x_M.shape)

    S_i = np.concatenate([x_K, x_M], axis=0)
    
    #print(S_i)
    G_i = np.concatenate([y_K, y_M], axis=0)
    init_OP, train_op, F, S, C, K, M = optimizeLogPost(G_i, kern)
    #print(G_i.shape)
    F.set_shape(G_i.shape)
    tf.assign(F, G_i, validate_shape=False)
    K_i = len(x_K)
    M_i = len(x_M)
    #print(K_i)
    with tf.Session() as sess:

        sess.run(init_OP)
        #sess.run(tf.initialize_all_variables())
        #sess.run(F)
        sess.run(train_op, feed_dict={S:S_i, K:K_i, M:M_i})
    return x_K, y_K, x_M, y_M
    
    
bern_p = 0.5
n_iter = 10
x_K, y_K = tf.constant(S_k, dtype=tf.float32), tf.constant(G_k, dtype=tf.float32)
print(y_K)
x_M, y_M = tf.Variable(tf.zeros((0,1)), validate_shape=False), tf.Variable(tf.zeros((0,1)), validate_shape=False)
ind = tf.Variable(0)
for i in range(n_iter):
    x_K, y_K, x_M, y_M = loop_step(n_iter,ind, x_K, y_K, x_M, y_M)
    x_M = tf.Variable(x_M, validate_shape=False)
    y_M = tf.Variable(y_M, validate_shape=False)


Tensor("Const_171:0", shape=(44, 1), dtype=float32)
Event inserted
Event deleted
(<tf.Tensor 'while_106/cond/cond_1/cond/Shape:0' shape=(?,) dtype=int32>, <tf.Tensor 'while_106/cond/cond_1/cond/Shape_1:0' shape=(?,) dtype=int32>)
Thinning completed
Locations found
((44, 1), (7, 1))


InvalidArgumentError: Expected begin[0] in [0, 1], but got 44
	 [[Node: Slice_97 = Slice[Index=DT_INT32, T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](Variable_302/read, Slice_97/begin, ones_97/shape)]]

Caused by op u'Slice_97', defined at:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/usr/lib/python2.7/dist-packages/zmq/eventloop/ioloop.py", line 162, in start
    super(ZMQIOLoop, self).start()
  File "/usr/lib/python2.7/dist-packag

In [ ]:
S = tf.placeholder(tf.float32)
C = tf.placeholder(tf.float32)
K = tf.placeholder(tf.int32)
M = tf.placeholder(tf.int32)
F = tf.Variable(np.zeros((1,1)), dtype=tf.float32)
N = K + M

prior_loss = 0.5 * tf.matmul(tf.transpose(F), tf.matmul(tf.matrix_inverse(C + 1e-6*tf.eye(N)), F))
prior_loss = tf.squeeze(prior_loss)
likelihood_loss = tf.reduce_sum(tf.log(tf.ones([K,1]) + tf.exp(-tf.slice(F, [0,0], [K,1]))))
likelihood_loss += tf.reduce_sum(tf.log(tf.ones([M,1]) + tf.exp(tf.slice(F, [K,0], [M,1]))))    
loss = prior_loss + likelihood_loss